In [1]:
class Enigma:
    def __init__(self, rotor_positions, rotor_mapping):
        self.rotor_positions = rotor_positions
        self.rotor_mapping = rotor_mapping
        self.plugboard = {}  # 플러그보드 설정용

    def set_plugboard(self, connections):
        """플러그보드 설정"""
        for connection in connections:
            a, b = connection
            self.plugboard[a] = b
            self.plugboard[b] = a

    def encode_letter(self, letter):
        """문자 하나를 암호화"""
        # 플러그보드 대체 적용
        if letter in self.plugboard:
            letter = self.plugboard[letter]

        # 로터를 통해 암호화
        for rotor in self.rotor_mapping:
            letter = rotor[letter]

        # 로터 회전
        self.rotate_rotors()
        return letter

    def rotate_rotors(self):
        """로터 회전"""
        self.rotor_positions = [(pos + 1) % 26 for pos in self.rotor_positions]

    def encrypt(self, message):
        """메시지 암호화"""
        encrypted_message = ''
        for letter in message:
            encrypted_message += self.encode_letter(letter)
        return encrypted_message

    def decode_letter(self, letter):
        """문자 하나를 복호화"""
        # 로터 역순으로 복호화
        for rotor in reversed(self.rotor_mapping):
            letter = [k for k, v in rotor.items() if v == letter][0]

        # 플러그보드 대체 다시 적용
        if letter in self.plugboard:
            letter = self.plugboard[letter]
        
        return letter

    def decrypt(self, message):
        """메시지 복호화"""
        decrypted_message = ''
        for letter in message:
            decrypted_message += self.decode_letter(letter)
        return decrypted_message


# 로터 설정 (간단한 매핑 예시)
rotor_mapping = [{
    'A': 'E', 'B': 'F', 'C': 'G',
    'D': 'H', 'E': 'I', 'F': 'J',
    'G': 'K', 'H': 'L', 'I': 'M',
    'J': 'N', 'K': 'O', 'L': 'P',
    'M': 'Q', 'N': 'R', 'O': 'S',
    'P': 'T', 'Q': 'U', 'R': 'V',
    'S': 'W', 'T': 'X', 'U': 'Y', 
    'V': 'Z', 'W': 'A', 
    'X': 'B',  'Y': 'C',  'Z': 'D'
}]

# 애니그마 기계 초기화
enigma = Enigma(rotor_positions=[0], rotor_mapping=rotor_mapping)

# 플러그보드 연결 설정 (선택 사항)
# enigma.set_plugboard([('A', 'B'), ('C', 'D')])

# 메시지 암호화
message = "HELLO"
encrypted_message = enigma.encrypt(message)
print(f"암호화된 메시지: {encrypted_message}")

# 메시지 복호화
decrypted_message = enigma.decrypt(encrypted_message)
print(f"복호화된 메시지: {decrypted_message}")

암호화된 메시지: LIPPS
복호화된 메시지: HELLO


In [2]:
import string
from itertools import permutations

class EnigmaMachine:
    def __init__(self, rotors, reflector, plugboard=None):
        self.rotors = rotors
        self.reflector = reflector
        self.plugboard = plugboard or {}

    def encode(self, message):
        encoded_message = ''
        for char in message:
            if char in string.ascii_uppercase:
                if char in self.plugboard:
                    char = self.plugboard[char]
                char = self.pass_through_rotors(char, forward=True)
                char = self.reflect(char)
                char = self.pass_through_rotors(char, forward=False)
                if char in self.plugboard:
                    char = self.plugboard[char]
                self.step_rotors()
                encoded_message += char
            else:
                encoded_message += char  # Non-alphabetic characters are unchanged
        return encoded_message

    def pass_through_rotors(self, char, forward):
        for rotor in (self.rotors if forward else reversed(self.rotors)):
            char = rotor.encode(char, forward)
        return char

    def reflect(self, char):
        return self.reflector[char]

    def step_rotors(self):
        self.rotors[0].step()
        for i in range(len(self.rotors) - 1):
            if self.rotors[i].at_notch():
                self.rotors[i + 1].step()

class Rotor:
    def __init__(self, wiring, notch):
        self.wiring = wiring
        self.notch = notch
        self.position = 0

    def encode(self, char, forward):
        index = (string.ascii_uppercase.index(char) + self.position) % 26
        if forward:
            return self.wiring[index]
        else:
            index = self.wiring.index(char)
            return string.ascii_uppercase[(index - self.position) % 26]

    def step(self):
        self.position = (self.position + 1) % 26

    def at_notch(self):
        return string.ascii_uppercase[self.position] == self.notch

def crack_enigma(encoded_message, known_plaintext):
    rotor_wirings = [
        "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
        "AJDKSIRUXBLHWTMCQGZNPYFVOE",
        "BDFHJLCPRTXVZNYEIWGAKMUSQO"
    ]
    reflector = {char: char2 for char, char2 in zip(string.ascii_uppercase, "YRUHQSLDPXNGOKMIEBFZCWVJAT")}
    for rotor_order in permutations(rotor_wirings, 3):
        for r1_position in range(26):
            for r2_position in range(26):
                for r3_position in range(26):
                    rotors = [Rotor(rotor_order[0], "Q"), Rotor(rotor_order[1], "E"), Rotor(rotor_order[2], "V")]
                    rotors[0].position = r1_position
                    rotors[1].position = r2_position
                    rotors[2].position = r3_position
                    enigma = EnigmaMachine(rotors, reflector)
                    decoded_message = enigma.encode(encoded_message)
                    if known_plaintext in decoded_message:
                        return f"Decoded Message: {decoded_message}, Rotor Positions: {r1_position, r2_position, r3_position}, Rotor Order: {rotor_order}"
    return "Failed to decode the message."

# Example encoded message and known part of the plaintext
encoded_message = "QWERTYUIOP"
known_plaintext = "HELLO"

result = crack_enigma(encoded_message, known_plaintext)
print(result)


Decoded Message: NXJSUHELLO, Rotor Positions: (0, 13, 11), Rotor Order: ('BDFHJLCPRTXVZNYEIWGAKMUSQO', 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'EKMFLGDQVZNTOWYHXUSPAIBRCJ')
